<a href="https://colab.research.google.com/github/gurtaransingh/scoreme_hackathon/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code optimised for Test3 file**

##Purpose:
This Python script extracts structured data from a PDF file and saves it into an Excel spreadsheet. It utilizes PyMuPDF for PDF handling and pandas for data manipulation.

##Features:
PDF Parsing: The script reads a specified PDF file (test3.pdf in this case) using PyMuPDF, extracting text from each page.

##Data Extraction:

**Field and Value Extraction:** It identifies fields and their corresponding values by splitting lines based on colon (:) where possible.

**Amount Extraction:** It extracts amounts with decimals and commas from lines using regular expressions.

**Amounts with "Dr":** It captures amounts followed by "Dr" for financial transactions.

**Transaction Type:** It identifies transaction types ('C' for Credit, 'T' for Debit) and extracts text following them.

**Important Dates:** It captures dates in the format DD-MMM-YYYY from the text.

**Transaction IDs:** It identifies transaction IDs, typically numeric sequences of 10 digits or more.

**Data Organization:** Extracted data is stored in a pandas DataFrame, which allows for easy manipulation and analysis.

**Output:** The processed data is saved into an Excel file (test3_extracted_text_final_v7.xlsx), preserving the structure and details extracted from the PDF.

##Usage:
Ensure Python environment has necessary libraries installed: fitz, pandas, and re.

Replace pdf_path variable with the path to your PDF file.

Run the script to extract data.

Check the generated Excel file for extracted and organized data (test3_extracted_text_final_v7.xlsx).

##Notes:
Ensure the PDF file is accessible and correctly specified in pdf_path.

The script uses regular expressions to handle varying formats of data entries (amounts, dates, transaction IDs, etc.).

Adjustments may be needed based on specific PDF formats and data extraction requirements.

In [5]:
import fitz  # PyMuPDF for PDF handling
import pandas as pd
import re  # Regular Expression module for pattern matching

# Replace with your actual PDF path
pdf_path = '/content/drive/MyDrive/HACKATHON/test3.pdf'

# Function to extract amounts with decimals and commas from a line
def extract_amount(line):
    # Regular expression pattern to match amounts with decimals and commas
    pattern = r'\b\d{1,3}(?:,?\d{3})*(?:\.\d{1,2})\b'
    amounts = re.findall(pattern, line)
    return amounts

# Function to extract amounts with "Dr" from a line
def extract_dr_amount(line):
    # Regular expression pattern to match amounts with "Dr" and decimals and commas
    pattern = r'\b\d{1,3}(?:,?\d{3})*(?:\.\d{1,2})Dr\b'
    dr_amounts = re.findall(pattern, line)
    return dr_amounts

# Function to extract "C" or "T" from a line
def extract_transaction_type(line):
    # Regular expression pattern to match "C" or "T"
    pattern = r'\b[C|T]\b'
    types = re.findall(pattern, line)
    return types

# Function to extract text after "C" or "T" from a line
def extract_text_after_ct(line):
    # Regular expression pattern to match text after "C" or "T"
    pattern = r'\b[C|T]\s+(.+)$'
    matches = re.findall(pattern, line)
    if matches:
        return matches[0].strip()
    else:
        return ''

# Open the PDF file
pdf_document = fitz.open(pdf_path)

# Initialize variables to store data
data = {
    'Field': [],
    'Value': [],
    'Amount': [],
    'Amount with Dr': [],
    'Transaction Type': [],
    'Text after CT': [],
    'Important Dates': [],
    'Transaction ID': []
}

# Iterate through all the pages and extract text
for page_num in range(pdf_document.page_count):
    page = pdf_document.load_page(page_num)
    text = page.get_text()

    # Split text by new lines to handle multiline fields
    lines = text.split('\n')
    for line in lines:
        # Ignore empty lines
        if line.strip() == '':
            continue

        # Try splitting by ':' to separate field and value
        if ':' in line:
            parts = line.split(':', 1)
            field = parts[0].strip()
            value = parts[1].strip()
        else:
            # If no ':', treat the entire line as the field (for multiline values)
            field = ''
            value = line.strip()

        # Extract amounts and amounts with "Dr" from both field and value
        amounts_field = extract_amount(field)
        amounts_value = extract_amount(value)
        dr_amounts_field = extract_dr_amount(field)
        dr_amounts_value = extract_dr_amount(value)

        # Combine extracted values while ensuring no duplicates
        amounts = list(set(amounts_field + amounts_value))
        dr_amounts = list(set(dr_amounts_field + dr_amounts_value))

        # Extract transaction type (C or T) from both field and value
        transaction_types_field = extract_transaction_type(field)
        transaction_types_value = extract_transaction_type(value)

        # Combine extracted types while ensuring no duplicates
        transaction_types = list(set(transaction_types_field + transaction_types_value))

        # Extract text after "C" or "T"
        text_after_ct = extract_text_after_ct(value)

        # Extract dates and transaction IDs from both field and value
        dates_field = re.findall(r'\b\d{2}-[A-Za-z]{3}-\d{4}\b', field)
        dates_value = re.findall(r'\b\d{2}-[A-Za-z]{3}-\d{4}\b', value)
        transaction_ids_field = re.findall(r'\b\d{10,}\b', field)
        transaction_ids_value = re.findall(r'\b\d{10,}\b', value)

        # Combine extracted values while ensuring no duplicates
        dates = list(set(dates_field + dates_value))
        transaction_ids = list(set(transaction_ids_field + transaction_ids_value))

        # Append to data dictionary
        data['Field'].append(field)
        data['Value'].append(value)
        data['Amount'].append(", ".join(amounts))
        data['Amount with Dr'].append(", ".join(dr_amounts))
        data['Transaction Type'].append(", ".join(transaction_types))
        data['Text after CT'].append(text_after_ct)
        data['Important Dates'].append(", ".join(dates))
        data['Transaction ID'].append(", ".join(transaction_ids))

# Close the PDF document
pdf_document.close()

# Create a DataFrame
df = pd.DataFrame(data)

# Save to Excel
excel_output_path = 'test3_extracted_text_final_v7.xlsx'
df.to_excel(excel_output_path, index=False)

print(f"Extracted and refined text saved to '{excel_output_path}'")


Extracted and refined text saved to 'test3_extracted_text_final_v7.xlsx'


# **Code optimised for Test5 File**

##Overview
This script extracts specific data from a PDF file and saves it into an Excel file. It uses the PyMuPDF library to read the PDF and regular expressions to parse and extract the required information.

##Features
Extracts and separates the first two columns from each line in the PDF.
Identifies and extracts dates, transaction texts, transaction IDs, and secondary dates from the PDF.
Stores the extracted data in a structured format and exports it to an Excel file.
##Requirements
* PyMuPDF
* pandas
* re (Regular Expressions module)


In [6]:
import fitz  # PyMuPDF
import pandas as pd
import re

# Replace with your actual PDF path
pdf_path = '/content/drive/MyDrive/HACKATHON/test5.pdf'

# Open the PDF file
pdf_document = fitz.open(pdf_path)

# Initialize variables to store data
data = {
    'Column 1': [],
    'Column 2': [],
    'Date': [],
    'Text': [],
    'Transaction ID': [],
    'Date 2': []
}

# Regex patterns
date_pattern = r'\d{2}/\d{2}/\d{2}'
text_pattern = r'[A-Z]+-\d+-[A-Z\s\d]+'
transaction_id_pattern = r'\d{12}'

# Iterate through all the pages and extract text
for page_num in range(pdf_document.page_count):
    page = pdf_document.load_page(page_num)
    text = page.get_text()

    lines = text.split('\n')
    for line in lines:
        if re.search(date_pattern, line):
            # Split by spaces and handle each part
            parts = line.split()
            # Column 1 and Column 2
            column_1 = parts[0]
            column_2 = parts[1]

            # Date, Text, Transaction ID
            date_match = re.search(date_pattern, line)
            text_match = re.search(text_pattern, line)
            transaction_id_match = re.search(transaction_id_pattern, line)

            date = date_match.group() if date_match else ''
            text = text_match.group() if text_match else ''
            transaction_id = transaction_id_match.group() if transaction_id_match else ''

            # Date 2
            date2_match = re.findall(date_pattern, line)
            date2 = date2_match[1] if len(date2_match) > 1 else ''

            # Append to data dictionary
            data['Column 1'].append(column_1)
            data['Column 2'].append(column_2)
            data['Date'].append(date)
            data['Text'].append(text)
            data['Transaction ID'].append(transaction_id)
            data['Date 2'].append(date2)

# Close the PDF document
pdf_document.close()

# Create a DataFrame
df = pd.DataFrame(data)

# Save to Excel
excel_output_path = 'test5_extracted_data.xlsx'
df.to_excel(excel_output_path, index=False)

print(f"Extracted data saved to '{excel_output_path}'")


Extracted data saved to 'test5_extracted_data.xlsx'
